In [7]:
import time
import tqdm #tqdm은 스테이터스 바 (진행 바)
from tqdm import tqdm_notebook
import pandas as pd
import numpy as np
from glob import glob
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import urllib.request
import time
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
# 크롬 드라이버 자동 업데이트
from webdriver_manager.chrome import ChromeDriverManager
# 브라우저 꺼짐 방지
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
# 불필요한 에러 메시지 없애기
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
service = Service(executable_path=ChromeDriverManager().install())

In [8]:
import warnings
warnings.filterwarnings(action = "ignore")

In [9]:
#웹 브라우저 열기
driver = webdriver.Chrome(options=chrome_options)
url = "https://tmacs.kotsa.or.kr/web/TG/TG200/TG2100S/Tg2102.jsp?mid=S1177"
driver.get(url)

In [39]:
#지자체(구별) 크롤링 엑셀 다운로드
jijace_list_raw = driver.find_element(By.ID, "jijace") #지자체 
jijace_list = jijace_list_raw.find_elements(By.TAG_NAME, "option") #지자체 옵션
jijace_names = [option.text for option in jijace_list] #구 이름 저장
jijace_names = jijace_names[1:]

search = driver.find_element(By.CSS_SELECTOR, "#content > div.cont_box > div.filterForm > div.btn_wrap.btn-2 > a.btn.btn_primary")
excel_btn = driver.find_element(By.CSS_SELECTOR, "#content > div.cont_box > div:nth-child(2) > div.btn_box.type02 > a.btn.exbtn")

for gu in tqdm_notebook(jijace_names):
    jijace_list_raw.send_keys(gu)
    time.sleep(3)
    search.click()
    time.sleep(2)
    excel_btn.click()
    time.sleep(2)

  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
season_files = glob("원본데이터/서울_계절별/교통사고 지표 현황*xls")

In [5]:
tmp_raw = []
for name in season_files:
    tmp = pd.read_excel(name, header = 1,usecols=[0, 1, 2, 3, 4, 8])
    tmp_raw.append(tmp)

seoul_season_raw = pd.concat(tmp_raw)

In [6]:
seoul_season_raw.rename(columns={'지자체': '구'}, inplace=True)

In [7]:
seoul_season_raw.dropna(inplace=True) #결측치 삭제

In [8]:
seoul_season_raw['연도'] = seoul_season_raw['연도'].astype(str).str.split('.').str[0]

In [11]:
# 피봇 테이블 직전 버전
seoul_season_raw.to_excel("./정제데이터/서울/서울_계절별_raw.xlsx") 

In [13]:
seoul_season = seoul_season_raw.pivot_table( index = ["시도", "구", "연도"], columns = "계절별", values=['발생건수', '사망자수'], aggfunc = np.sum)
seoul_season

발생건수                                 사망자수                    \
계절별         가을(9~11월) 겨울(12~2월) 봄(3~5월) 여름(6~8월) 가을(9~11월) 겨울(12~2월) 봄(3~5월)   
시도 구   연도                                                                      
서울 강남구 2018     838.0     857.0   849.0    915.0       6.0       5.0     4.0   
       2019    1021.0     858.0   895.0    948.0       4.0       6.0     2.0   
       2020     926.0     887.0   876.0   1063.0       4.0       5.0     7.0   
       2021     972.0     952.0   934.0    962.0       2.0       4.0     1.0   
       2022     978.0     852.0   890.0    989.0       3.0       3.0     7.0   
...               ...       ...     ...      ...       ...       ...     ...   
   중랑구 2018     417.0     374.0   388.0    386.0       2.0       2.0     4.0   
       2019     474.0     366.0   386.0    394.0       4.0       0.0     4.0   
       2020     416.0     362.0   331.0    338.0       1.0       1.0     2.0   
       2021     339.0     327.0   367.0    340.0       1.0       2.0     2.0   
       2022     352.0     318.0   363.0    368.0       5.0       2.0     2.0   

                      
계절별         여름(6~8월)  
시도 구   연도             
서울 강남구 2018      5.0  
       2019      2.0  
       2020      1.0  
       2021      3.0  
       2022      3.0  
...              ...  
   중랑구 2018      2.0  
       2019      4.0  
       2020      0.0  
       2021      2.0  
       2022      1.0  

[125 rows x 8 columns]

In [14]:
new_columns = []

for num in range(int(len(seoul_season.columns)/2)):
    new_columns.append('발생_'+seoul_season.columns[num][1])

for num in range(int(len(seoul_season.columns)/2)):
    new_columns.append('사망_'+seoul_season.columns[num][1])

seoul_season.columns = new_columns

seoul_season.reset_index(inplace=True)

In [15]:
seoul_season.to_excel("./정제데이터/서울/서울_계절별.xlsx")

In [16]:
seoul_season

,시도,구,연도,발생_가을(9~11월),발생_겨울(12~2월),발생_봄(3~5월),발생_여름(6~8월),사망_가을(9~11월),사망_겨울(12~2월),사망_봄(3~5월),사망_여름(6~8월)
0,서울,강남구,2018,838.0,857.0,849.0,915.0,6.0,5.0,4.0,5.0
1,서울,강남구,2019,1021.0,858.0,895.0,948.0,4.0,6.0,2.0,2.0
2,서울,강남구,2020,926.0,887.0,876.0,1063.0,4.0,5.0,7.0,1.0
3,서울,강남구,2021,972.0,952.0,934.0,962.0,2.0,4.0,1.0,3.0
4,서울,강남구,2022,978.0,852.0,890.0,989.0,3.0,3.0,7.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...
120,서울,중랑구,2018,417.0,374.0,388.0,386.0,2.0,2.0,4.0,2.0
121,서울,중랑구,2019,474.0,366.0,386.0,394.0,4.0,0.0,4.0,4.0
122,서울,중랑구,2020,416.0,362.0,331.0,338.0,1.0,1.0,2.0,0.0
123,서울,중랑구,2021,339.0,327.0,367.0,340.0,1.0,2.0,2.0,2.0


### 데이터 정제 -자동화 버전

In [40]:
#지역 변수 선언
region_list = ["서울", "부산", "대구", "인천", "광주", "대전", "울산", "세종", "경기", "강원", "충북", "충남", "전북", "전남", "경북", "경남", "제주" ]
detail = "계절별"

for region in region_list:
    season_files = glob(f"원본데이터/{detail}/{region}_{detail}/교통사고 지표 현황*xls")
    tmp_raw = []

    for name in season_files:
        tmp = pd.read_excel(name, header = 1,usecols=[0, 1, 2, 3, 4, 8])
        tmp_raw.append(tmp)

    season_raw = pd.concat(tmp_raw)

    season_raw.rename(columns={'지자체': '구'}, inplace=True)

    season_raw = season_raw.dropna() #결측치 삭제

    season_raw['연도'] = season_raw['연도'].astype(str).str.split('.').str[0] #연도 str화

    # 피봇 테이블 직전 버전
    season_raw.to_excel(f"./정제데이터/{region}/{region}_{detail}_raw.xlsx") 

    season = season_raw.pivot_table( index = ["시도", "구", "연도"], columns = f"{detail}", values=['발생건수', '사망자수'], aggfunc = np.sum)
    season

    #평탄화 작업
    new_columns = []

    for num in range(int(len(season.columns)/2)):
        new_columns.append('발생_'+season.columns[num][1])

    for num in range(int(len(season.columns)/2)):
        new_columns.append('사망_'+season.columns[num][1])

    season.columns = new_columns

    season.reset_index(inplace=True)

    # 최종 버전
    season.to_excel(f"./정제데이터/{region}/{region}_{detail}.xlsx") 